### 환경 설정

In [4]:
from dotenv import load_dotenv
from rich import print as rprint


load_dotenv()


True

# Models
2가지 case로 사용
1. 에이전트에서 바로 사용하거나
2. 모델만 독립적으로 사용

- 주로 openai나 gemini 사용

## 독립 모델 : `init_chat_model`
or
자체적으로 제공되는 클래스 사용도 가능(- 기존 사용 방법이지만 굳이?)
- 아 좀 더 추가적인 파라미터를 원한다면 사용



그래도 이제 통합해서 모델을 만들 수 있어서 훨씬 편함. 원래는 다 따로따로 모델마다 다르게 가져와서 만들었어야 했음

여러 패턴들이 있음

### 1. 고정(non-configurable) 모델
-  코드에서 모델을 정해놓고 쓰는 패턴

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    # Kwargs passed to the model:
    temperature=1.0,
    timeout=30, # 요청을 취소하기 전에 model의 응답을 기다리는 최대 시간(s)
    max_tokens=500, 
    max_retries=3, # 요청이 실패할 경우 재시도 횟수
)

In [6]:
response = model.invoke("고양이는 귀여워?")
rprint(response)



AIMessage(
    content='네, 고양이는 일반적으로 매우 귀엽다고 여겨집니다. 😻\n\n고양이의 귀여움은 여러 가지 요소에서 
비롯됩니다.\n\n*   **외모:** 동그란 얼굴, 큰 눈, 쫑긋한 귀, 부드러운 털 등은 많은 사람들에게 사랑스러운 느낌을 
줍니다.\n*   **행동:** 장난스럽게 뛰어다니거나, 몸을 비비거나, 골골송을 부르는 모습, 잠자는 모습 등 다양한 행동들이
귀엽게 느껴집니다.\n*   **성격:** 때로는 도도하지만, 애교를 부리거나 기대는 모습은 보호자에게 특별한 감정을 
불러일으킵니다.\n*   **표정:** 감정을 드러내는 듯한 눈빛이나 표정 변화 또한 귀여움을 더합니다.\n\n물론 사람마다 
선호하는 동물이나 귀엽다고 느끼는 기준은 다를 수 있지만, 전 세계적으로 고양이는 가장 인기 있는 반려동물 중 
하나이며, 많은 사람들이 고양이를 보며 귀엽다고 느낍니다.\n\n혹시 고양이에 대해 특별히 궁금한 점이 있으신가요?',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--cfabfd31-e84c-413e-93b7-a55176beaa27-0',
    usage_metadata={
        'input_tokens': 8,
        'output_tokens': 246,
        'total_tokens': 254,
        'input_token_details': {'cache_read': 0}
    }
)

### 2. Configurable 모델

#### 2-1. 부분 설정 가능 모델
- model을 안 주고, 나중에 config로 어떤 모델 쓸 지 결정

In [8]:
config_model = init_chat_model(temperature=0.5)

config_model.invoke(
    "고양이는 귀여워?",
    config={"configurable": {"model": "google_genai:gemini-2.5-flash-lite"}}
)

rprint(response)


AIMessage(
    content='네, 고양이는 일반적으로 매우 귀엽다고 여겨집니다. 😻\n\n고양이의 귀여움은 여러 가지 요소에서 
비롯됩니다.\n\n*   **외모:** 동그란 얼굴, 큰 눈, 쫑긋한 귀, 부드러운 털 등은 많은 사람들에게 사랑스러운 느낌을 
줍니다.\n*   **행동:** 장난스럽게 뛰어다니거나, 몸을 비비거나, 골골송을 부르는 모습, 잠자는 모습 등 다양한 행동들이
귀엽게 느껴집니다.\n*   **성격:** 때로는 도도하지만, 애교를 부리거나 기대는 모습은 보호자에게 특별한 감정을 
불러일으킵니다.\n*   **표정:** 감정을 드러내는 듯한 눈빛이나 표정 변화 또한 귀여움을 더합니다.\n\n물론 사람마다 
선호하는 동물이나 귀엽다고 느끼는 기준은 다를 수 있지만, 전 세계적으로 고양이는 가장 인기 있는 반려동물 중 
하나이며, 많은 사람들이 고양이를 보며 귀엽다고 느낍니다.\n\n혹시 고양이에 대해 특별히 궁금한 점이 있으신가요?',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--cfabfd31-e84c-413e-93b7-a55176beaa27-0',
    usage_metadata={
        'input_tokens': 8,
        'output_tokens': 246,
        'total_tokens': 254,
        'input_token_details': {'cache_read': 0}
    }
)

#### 2-2. 기본값이 있는 완전 설정 가능 모델
- 기본값을 쓰되, 필요할 때 런타임에서 값을 바꾸는 패턴

In [10]:
config_model = init_chat_model(
    model="google_genai:gemini-2.5-flash-lite",
    configurable_fields="any", # 나중에 어떤 값들을 바꿀 수 있는지에 대한 목록
        # any : 모든 필드 값들을 바꿀 수 있음
        # 튜플 리스트 : 특정 필드만 바꿀 수 있음
    config_prefix="foo", # 여러 모델이 있을 때, 각 모델이 config에서 쓸 키 이름 접두사 정의
    temperature=0.5,
)

# 기본값 사용
response1 = config_model.invoke("고양이는 귀여워?")

# 런타임에서 모델/온도 변경
response2 = config_model.invoke(
    "고양이는 귀여워?",
    config={"configurable": {
        "foo_model": "google_genai:gemini-2.5-pro",
        "foo_temperature": 1,
        }},
)

rprint(response1)
rprint(response2)

AIMessage(
    content='네, 고양이는 정말 귀엽습니다! 많은 사람들이 고양이의 동글동글한 얼굴, 반짝이는 눈, 부드러운 털, 그리고
애교 넘치는 행동에 매력을 느낍니다.\n\n고양이가 귀엽다고 느껴지는 이유는 여러 가지가 있습니다.\n\n*   **외모:** 큰 
눈, 동그란 얼굴, 쫑긋한 귀, 앙증맞은 코와 입 등 아기 같은 외모는 본능적으로 귀엽다는 느낌을 줍니다.\n*   
**행동:**\n    *   **애교:** 몸을 비비거나, 골골송을 부르거나, 꾹꾹이를 하는 행동은 사람에게 친밀감과 애정을 
표현하는 것으로 느껴져 귀엽게 보입니다.\n    *   **호기심:** 주변을 탐색하고 장난감에 집중하는 모습은 순수하고 
사랑스럽게 느껴집니다.\n    *   **잠자는 모습:** 편안하게 늘어져 자거나, 웅크리고 자는 모습은 평화롭고 
귀엽습니다.\n    *   **움직임:** 살금살금 걷거나, 잽싸게 뛰어다니거나, 점프하는 모습 등 민첩하고 우아한 움직임도 
매력적입니다.\n*   **소리:** 야옹거리는 소리, 골골송 등 고양이 특유의 소리도 귀엽다고 느끼는 사람들에게는 매력 
포인트가 됩니다.\n\n물론 귀여움은 주관적인 느낌이기 때문에 모든 사람이 똑같이 느낄 수는 없겠지만, 일반적으로 
고양이는 귀여운 동물로 널리 인식되고 사랑받고 있습니다.\n\n혹시 고양이의 어떤 점이 귀엽다고 생각하시나요? 아니면 
고양이 사진이나 영상을 보면 귀엽다고 느껴지시나요?',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--c55a8d90-614b-4d81-a564-3d629ed6aa33-0',
    usage_metadata={
        'input_tokens': 8,
        'output_tokens': 388,
        'total_tokens': 396,
        'input_token_details': {'cache_read': 0}
    }
)

AIMessage(
    content="네, 물론입니다! 고양이는 정말 귀엽죠. 많은 사람들이 고양이를 귀엽다고 생각하는 데에는 여러 가지 이유가
있습니다.\n\n**1. 외모적인 특징**\n*   **크고 동그란 눈:** 얼굴에 비해 큰 눈은 아기 같은 느낌을 주어 보호 본능을 
자극합니다.\n*   **부드러운 털:** 보기만 해도 폭신하고 부드러워 보여서 만지고 싶은 충동을 일으킵니다.\n*   
**말랑말랑한 젤리 발바닥:** 분홍색이나 검은색의 작은 발바닥은 고양이의 가장 큰 매력 포인트 중 하나입니다.\n*   
**작은 입과 코:** 오밀조밀 모여있는 입과 코가 사랑스러움을 더합니다.\n\n**2. 사랑스러운 행동**\n*   **골골송:** 
기분이 좋을 때 내는 '그르릉'거리는 소리는 듣는 사람의 마음을 편안하게 해줍니다.\n*   **꾹꾹이:** 어미 젖을 빨던 
습성이 남아 앞발로 꾹꾹 누르는 행동은 애정 표현의 하나로 매우 귀엽습니다.\n*   **몸 비비기:** 신뢰와 애정을 담아 
사람의 다리나 몸에 자신의 몸을 비비는 행동은 마음을 녹입니다.\n*   **엉뚱한 모습:** 상자만 보면 들어가려 하거나, 
갑자기 우다다 뛰어다니는 등 예측할 수 없는 엉뚱한 행동들이 웃음을 줍니다.\n\n**3. '츤데레' 같은 성격**\n무심한 
척하다가도 슬쩍 다가와 애교를 부리는 '밀고 당기기' 같은 모습이 또 다른 매력으로 다가옵니다.\n\n물론 귀여움을 느끼는
기준은 사람마다 다르겠지만, 고양이는 많은 사람들이 사랑할 수밖에 없는 다양한 매력을 가진 동물임이 틀림없습니다. 
😊\n\n혹시 고양이를 키우시나요? 어떤 점이 가장 귀엽다고 생각하세요?",
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-pro',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--d15fc7c3-e999-4d48-9f67-ddff4d93828a-0',
    usage_metadata={
        'input_tokens': 8,
        'output_tokens': 1727,
        'total_tokens': 1735,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 1282}
    }
)

#### 2-3. 커스텀
- 이거는 되는 게 아닌것같은데... 다시 확인 필요
- 아! 랭스미스 추적할 때 필요!

In [31]:
# # 구성을 사용한 호출
# response = model.invoke(
#     "안녕! 반가워.",
#     config={
#         "run_name": "greetings",  # 이 실행의 커스텀 이름
#         "tags": ["hi", "hello"],  # 분류를 위한 태그
#         "metadata": {"user_id": "teddy"},  # 커스텀 메타데이터
#     },
# )

# rprint(response)

AIMessage(
    content='안녕하세요! 저도 반가워요. 무엇을 도와드릴까요? 😊',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--c6075e55-5a33-42dc-b3e2-597d55693c11-0',
    usage_metadata={
        'input_tokens': 8,
        'output_tokens': 16,
        'total_tokens': 24,
        'input_token_details': {'cache_read': 0}
    }
)

실제 사용 예시

In [ ]:
llm = init_chat_model(
    model="google_genai:gemini-2.5-flash",
    temperature=0.5,
    configurable_fields=("model", "temperature", "max_tokens", "max_retries"),
    config_prefix="llm",
)

dev_config = {
    "configurable": {
        "llm_model": "google_genai:gemini-2.5-flash-lite",
        "llm_temperature": 1,
        "llm_max_tokens": 512,
    }
}

prod_config = {
    "configurable": {
        "llm_model": "google_genai:gemini-2.5-pro",
        "llm_temperature": 0.1,
        "llm_max_tokens": 1024,
    }
}

llm.invoke(
    "고양이는 귀여워?",
    config=dev_config,
)

llm.invoke(
    "고양이는 귀여워?",
    config=prod_config,
)

### Stream

- 아직 크게 쓸 곳이 없음

### batch
- 이것도 아직은 쓰는 곳 없긴한데, 곧 쓸듯

### 토큰 사용량 추적

In [33]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

model_1 = init_chat_model(model="google_genai:gemini-2.5-flash-lite")

# 콜백 핸들러를 사용하여 토큰 사용량 추적
callback = UsageMetadataCallbackHandler()

# config에 콜백 핸들러를 추가하여 토큰 사용량 추적
result_1 = model_1.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

{'gemini-2.5-flash-lite': {'input_tokens': 2,
  'output_tokens': 11,
  'total_tokens': 13,
  'input_token_details': {'cache_read': 0}}}

In [34]:
for i in range(3):
    model_1.invoke("Hello", config={"callbacks": [callback]})

    print("토큰 사용량은 callback.usage_metadata에 누적됩니다.")
    print(callback.usage_metadata)

토큰 사용량은 callback.usage_metadata에 누적됩니다.
{'gemini-2.5-flash-lite': {'output_tokens': 21, 'total_tokens': 25, 'input_tokens': 4, 'input_token_details': {'cache_read': 0}}}
토큰 사용량은 callback.usage_metadata에 누적됩니다.
{'gemini-2.5-flash-lite': {'output_tokens': 31, 'total_tokens': 37, 'input_tokens': 6, 'input_token_details': {'cache_read': 0}}}
토큰 사용량은 callback.usage_metadata에 누적됩니다.
{'gemini-2.5-flash-lite': {'output_tokens': 41, 'total_tokens': 49, 'input_tokens': 8, 'input_token_details': {'cache_read': 0}}}


## Tool calling
- model이 tool을 사용할 수 있도록 설정
- 딱 모델이 툴을 사용하겠다고 인지만 하고, 그걸 활용해서 답변까진 x 
    - `AIMessage`에서 `content`가 공백
    - `function_call`에서 어느 도구 호출한지 나옴 

### `bind_tools`

In [15]:
from langchain.chat_models import init_chat_model
from langchain.tools import tool


model = init_chat_model(
    "google_genai:gemini-2.5-flash-lite",
    # Kwargs passed to the model:
    temperature=1.0,
    timeout=30, # 요청을 취소하기 전에 model의 응답을 기다리는 최대 시간(s)
    max_tokens=500, 
    max_retries=3, # 요청이 실패할 경우 재시도 횟수
)

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

for tool_call in response.tool_calls:
    # View tool calls made by the model
    rprint(f"Tool: {tool_call['name']}")
    rprint(f"Args: {tool_call['args']}")

Tool: get_weather

Args: {'location': 'Boston'}

In [16]:
rprint(response)

AIMessage(
    content='',
    additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "Boston"}'}},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--fc0c1c73-3c92-49aa-ae21-e543564ff07f-0',
    tool_calls=[
        {
            'name': 'get_weather',
            'args': {'location': 'Boston'},
            'id': '405e9953-0e5c-4048-9985-507a7389c271',
            'type': 'tool_call'
        }
    ],
    usage_metadata={
        'input_tokens': 49,
        'output_tokens': 15,
        'total_tokens': 64,
        'input_token_details': {'cache_read': 0}
    }
)

### `pydantic` 모델

In [30]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


model_with_tools = model.bind_tools([GetWeather])
response = model_with_tools.invoke("서울(Seoul) 날씨는 어때?")
rprint(response)

AIMessage(
    content='US cities like "San Francisco, CA" are supported. Please provide the location in the correct format.',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.5-flash-lite',
        'safety_ratings': [],
        'model_provider': 'google_genai'
    },
    id='lc_run--b2ef08af-73f1-40ec-bf97-202942cd8d24-0',
    usage_metadata={
        'input_tokens': 62,
        'output_tokens': 21,
        'total_tokens': 83,
        'input_token_details': {'cache_read': 0}
    }
)

## Structured Output
- 기존과 동일하므로 pass